In [1]:
import numpy as np
import pandas as pd

In [2]:
import tensorflow as tf
import tensorflow.keras as k
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPool2D, Flatten, Reshape

In [3]:
import silence_tensorflow

In [4]:
from keras_bed_sequence import BedSequence
from keras_mixed_sequence import MixedSequence

# Data

In [5]:
from mendelian_snv_prediction import get_data

In [6]:
train, test = get_data(
    "./mendelian_snv_200.csv.gz",
    assembly="hg19",
    batchsize=128,
    head_threshold=1e6,
    max_wiggle_size=150, 
    wiggles=10,
)

0.999641


/home/zom/anaconda3/lib/python3.7/site-packages/ucsc_genomes_downloader/utils/wiggle_bed_regions.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bed.chromStart[bed.chromStart < 0] = 0


# Model

In [7]:
from mendelian_snv_prediction import get_model

In [10]:
model = get_model(
    windows_size=200,
    balancing_loss_weigth=1e5)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 200, 4)]          0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 200, 4, 1)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 200, 4, 128)       4736      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 66, 4, 128)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 66, 4, 64)         98368     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 33, 4, 64)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 33, 4, 64)         1235

# Test

In [11]:
history = model.fit_generator(
    generator=train,
    steps_per_epoch=train.steps_per_epoch,
    validation_data=test,
    validation_steps=test.steps_per_epoch,
    epochs=100,
    verbose=1,
    use_multiprocessing=True,
    shuffle=True
).history

Epoch 1/100
5489/5489 [==============================] - 612s 111ms/step - loss: 15663.0899 - binary_accuracy: 0.0098 - auc_1: 0.0039 - val_loss: 177.4273 - val_binary_accuracy: 3.6000e-04 - val_auc_1: 3.6000e-04
Epoch 2/100
5489/5489 [==============================] - 623s 114ms/step - loss: 14991.9688 - binary_accuracy: 0.0039 - auc_1: 0.0039 - val_loss: 177.4273 - val_binary_accuracy: 3.6000e-04 - val_auc_1: 3.6000e-04
Epoch 3/100
2799/5489 [==============>...............] - ETA: 4:44 - loss: 15921.1234 - binary_accuracy: 0.0039 - auc_1: 0.0039

Process Keras_worker_ForkPoolWorker-65:
Traceback (most recent call last):
  File "/home/zom/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/zom/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/zom/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/zom/anaconda3/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/home/zom/anaconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/zom/anaconda3/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/zom/anaconda3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: 